In [1]:
from premise import *
import bw2data
bw2data.projects.set_current("ei39")

In [2]:
scenarios = [
    {"model":"remind", "pathway":"SSP1-PkBudg500", "year":2030}, # optimistic renewable scenario, 2.3-2.8 °C
    {"model":"image", "pathway":"SSP2-RCP26", "year":2030}, # historical development, Paris Agreement, 1.6-1.8 °C
    {"model":"remind", "pathway":"SSP5-NPi", "year":2030}, # optimistic fossil, National policies, 4.0°C
]

In [3]:
ndb = NewDatabase(
        scenarios = scenarios,
        source_db= "ecoinvent cutoff 3.9.1",
        source_version="3.9.1",
        #source_type=source,
        #source_file_path=f"/Users/romain/Documents/ecoinvent {ei}_cutoff_ecoSpold02/datasets" if source == "ecospold" else None, # <-- this is NEW
        #source_version=ei,
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        #system_model="cutoff",
        #system_args=combinations[0]
)

premise v.(1, 5, 9)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
#ndb.update_electricity()
ndb.update_all()
#ndb.update_metals()

`update_all()` will skip the following steps:
update_two_wheelers(), update_cars(), and update_buses()
If you want to update these steps, please run them separately afterwards.
Extracted 1 worksheets in 7.96 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Done!
Done!
Extracted 1 worksheets in 7.48 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Done!
Done!
Extracted 1 worksheets in 7.44 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_da

In [6]:
for ds in ndb.database:
    #for e in ds["exchanges"]:
    #    if e["name"] == "methane, from electrochemical methanation, with carbon from atmospheric carbon dioxide capture":
    #        print(ds["name"], ds["location"])
    if ds["name"] == "market for clinker":
        print(ds["name"], ds["location"], ds["reference product"], ds["unit"], ds["code"])

market for clinker IND clinker kilogram 0472b19501d041f7ad0ae286c1a6b92f
market for clinker KOR clinker kilogram 481ed3a3b61645918f822c868bbeb88f
market for clinker RCAM clinker kilogram ff183ad032504f278c47b2eb502a0d56
market for clinker CAZ clinker kilogram bd84e839820844afac5738ae22f08cc0
market for clinker MEX clinker kilogram ec0ebd18de404f47826ffb369ff51443
market for clinker BR clinker kilogram cd27d6a65cdbd4a3815eeff6a11a518a
market for clinker EAF clinker kilogram 62d5ce1a205344afb48a430d5d4329b2
market for clinker WAF clinker kilogram 9bc6b5d847684d9281ad48514f77341b
market for clinker EUR clinker kilogram 5c88f031333b43ce99b026ed684ccb27
market for clinker CEU clinker kilogram e5ec3972316b40c787eb3d441eea8980
market for clinker UKR clinker kilogram d33ef76e6d9642ae80a4d47b36db2734
market for clinker OCE clinker kilogram cc15f5837af34fefb6896d57f255fff0
market for clinker CO clinker kilogram 10fd11246ec8eb9637db4e2769970c09
market for clinker TUR clinker kilogram a1de001196a7

In [5]:
ndb.write_superstructure_db_to_brightway("sebastian_test")

Building superstructure database...
Dropped 0 duplicate(s).
Scenario difference file exported to /Users/romain/Github/premise/dev/export/scenario diff files!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:12


Title: Writing activities to SQLite3 database:
  Started: 08/04/2023 13:44:44
  Finished: 08/04/2023 13:45:56
  Total time elapsed: 00:01:12
  CPU %: 97.20
  Memory %: 25.82
Created database: sebastian_test
Generate scenario report.
Report saved under /Users/romain/Github/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/Github/premise/dev.


In [12]:
('market for platinum', 'platinum', 'World', 'kilogram') in ndb.modified_datasets[("image", "SSP2-Base", 2050)]["created"]

True

In [13]:
ndb.scenarios[0].

AttributeError: 'NewDatabase' object has no attribute 'cache'

In [10]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"] == "market for platinum" and e["product"] == "platinum":
            print(ds["name"], ds["location"], e["name"], e["location"])

petrol production, unleaded, petroleum refinery operation IN market for platinum GLO
pitch production, petroleum refinery operation RoW market for platinum World
reformate production, petroleum refinery operation RoW market for platinum World
sulfur production, petroleum refinery operation PE market for platinum GLO
petroleum coke production, petroleum refinery operation RoW market for platinum World
heavy fuel oil production, petroleum refinery operation Europe without Switzerland market for platinum GLO
electricity production, medium voltage, petroleum refinery operation IN market for platinum GLO
base oil production, petroleum refinery operation BR market for platinum GLO
electricity production, medium voltage, petroleum refinery operation CO market for platinum GLO
white spirit production, petroleum refinery operation RoW market for platinum World
white spirit production, petroleum refinery operation Europe without Switzerland market for platinum GLO
internal combustion engine prod

In [5]:
import pprint
for ds in ndb.scenarios[0]["database"]:
    if ds["name"].lower() == "lithium carbonate, precipitated":
        print(ds["name"], ds["location"])

In [4]:
from premise.activity_maps import InventorySet
inv = InventorySet(ndb.database, "3.9")

In [5]:
inv.generate_metals_map()

{'Dysprosium': {'Dysprosium'},
 'Neodymium': {'Neodymium, 4% in bastnasite, 0.4% in crude ore'},
 'Nickel': {'Nickel',
  'Nickel, 1.13% in sulfide, Ni 0.76% and Cu 0.76% in crude ore',
  'Nickel, 1.98% in silicates, 1.04% in crude ore',
  'Nickel, Ni 2.5E+0%, in mixed ore'},
 'Manganese': {'Manganese',
  'Manganese, 35.7% in sedimentary deposit, 14.2% in crude ore'},
 'Yttrium': {'Yttrium'},
 'Vanadium': {'Vanadium'},
 'Cobalt': {'Cobalt', 'Cobalt, Co 5.0E-2%, in mixed ore'},
 'Lithium': {'Lithium', 'Lithium, 0.15% in brine'},
 'Sulfur': {'Sulfur'},
 'Platinum': {'Platinum', 'Platinum, Pt 4.7E-7%, in mixed ore'},
 'Cerium': {'Cerium', 'Cerium, 24% in bastnasite, 2.4% in crude ore'},
 'Lanthanum': {'Lanthanum',
  'Lanthanum, 7.2% in bastnasite, 0.72% in crude ore'},
 'Zirconium': {'Zirconium', 'Zirconium, 50% in zircon, 0.39% in crude ore'},
 'Gadolinium': {'Gadolinium',
  'Gadolinium, 0.15% in bastnasite, 0.015% in crude ore'},
 'Scandium': {'Scandium'},
 'Strontium': {'Strontium'},
 '

In [6]:
inv.generate_powerplant_map()

{'Biomass CHP': {'heat and power co-generation, wood chips, 2000 kW',
  'heat and power co-generation, wood chips, 2000 kW, state-of-the-art 2014',
  'heat and power co-generation, wood chips, 6667 kW',
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014, renewable energy products'},
 'Biomass CHP CCS': {'electricity production, at co-generation power plant/wood, post, pipeline 200km, storage 1000m',
  'electricity production, at co-generation power plant/wood, post, pipeline 400km, storage 3000m'},
 'Biomass ST': {'electricity production, at wood burning power plant 20 MW, truck 25km, no CCS'},
 'Biomass IGCC CCS': {'electricity production, at BIGCC power plant, pre, pipeline 200km, storage 1000m',
  'electricity production, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m'},
 'Biomass IGCC': {'electricity production, at BIGCC power plant, no CCS'},
 

In [8]:
inv.generate_daccs_map()

{'dac_solvent': {'carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2'},
 'dac_sorbent': {'carbon dioxide, captured from atmosphere, with a sorbent-based direct air capture system, 100ktCO2'},
 'daccs_solvent': {'carbon dioxide, captured from atmosphere and stored, with a solvent-based direct air capture system, 1MtCO2'},
 'daccs_sorbent': {'carbon dioxide, captured from atmosphere and stored, with a sorbent-based direct air capture system, 100ktCO2'}}

In [5]:
ndb.write_db_to_brightway("test_metals")

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
One or multiple duplicates detected. Removing them...
- check for values format...
- relinking exchanges...
Done!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:31


Title: Writing activities to SQLite3 database:
  Started: 06/19/2023 12:11:07
  Finished: 06/19/2023 12:11:38
  Total time elapsed: 00:00:31
  CPU %: 86.50
  Memory %: 11.62
Created database: test_metals
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev.


In [ ]:
products = [
    'lightweighting, glider',
    'auxilliary battery inputs, for road vehicle',
    'battery cell, Li-ion, NMC622',
    'auxilliary electronics inputs, for fuel cell electric vehicle',
    'auxilliary material inputs, for fuel cell electric vehicle',
    'fuel cell stack, for fuel cell electric vehicle',
    'carbon fiber, weaved',
    'tank liner, polyethylene, high density',
    'transformer and rectifier unit, for electrolyser',
    'fuel tank, for diesel road vehicle',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed hydrogen gas, 700bar',
    'fuel tank, for compressed natural gas, 200 bar',
    'charger, level 3, plugin, 200 kW, for electric vehicle',
    'gearbox, for lorry',
    'transmission, for lorry',
    'suspension, for lorry',
    'power electronics, for lorry',
    'auxilliary material inputs, for electric lorry',
    'retarder, for lorry',
    'cabin, for lorry',
    'frame, blanks and saddle, for lorry',
    'tires and wheels, for lorry',
    'auxilliary material inputs, for hydrid electric lorry',
    'exhaust system, for lorry',
    'internal combustion engine, for lorry',
    'lead acid battery, for lorry',
    'charger, level 3, with pantograph, 450 kW, for electric vehicle',
    'overhead lines, for electric vehicle',
    'battery cell, Li-ion, LTO',
    'charging station, 3kW',
    'battery cell,  Li-ion, LFP',
    'battery cell, Li-ion, NCA',
    'assembly operation, for lorry'
]

print(len(products))

acts = [a for a in bw.Database("brian") if a["reference product"] in products]

In [ ]:
len(acts)

In [ ]:
[a["reference product"] for a in acts]

In [ ]:
import bw2data
bw2io.BW2Package.export_obj(obj=bw.Database("brian"), filename="db_for_brian.bw2package")

In [ ]:
import bw2data
bw2io.BW2Package.import_file("db_for_brian.bw2package")

In [ ]:
fp = "/Users/romain/Library/Application Support/Brightway3/ei39.f1469ad5eb0a071dc60323f6e0fce980/output/lci-brian.xlsx"
import bw2io

i = bw2io.ExcelImporter(fp)
i.apply_strategies()

In [ ]:
ei_acts = list(set([(a["name"], a["reference product"]) for a in bw.Database("ecoinvent 3.9.1 cutoff")]))
print(len(ei_acts))

In [ ]:
for ds in i.data:
    if (ds["name"], ds["reference product"]) in ei_acts:
        i.data.remove(ds)

In [ ]:
for ds in i.data:
    if (ds["name"], ds["reference product"]) not in ei_acts:
        print(ds["name"])

In [ ]:
len(i.data)

In [ ]:
for ds in i.data:
    print(ds["name"])

In [ ]:
i.match_database(fields=["name", "unit", "location"])
i.match_database("ecoinvent 3.9.1 cutoff", fields=["name", "reference product", "location", "unit"])
i.match_database("biosphere3", fields=["name", "categories", "unit"])

In [ ]:
i.statistics()

In [ ]:
list(i.unlinked)[-1]

In [ ]:
bw.get_activity(('brian', '360c5e612a154ebb8cee2650bcc1bc75_copy1'))

In [ ]:
for f in bw.Database("biosphere3"):
    if f["code"] == "e030108f-2125-4bcb-a73b-ad72130fcca3":
        print(f)

In [ ]:
bw.projects.set_current("ei39")

In [ ]:
l = []

for f in bw.Database("biosphere3"):
    l.append([f["name"], f["categories"][0], f["categories"][1] if len(f["categories"])>1 else "unspecified", f["unit"], f["code"]])

In [ ]:
import csv

with open("flows_biosphere_39.csv", "w", newline="", ) as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(l)

In [ ]:
ndb.scenarios[1]["iam data"].fuel_markets.sel(region="NEU").variables.values

In [ ]:
import numpy as np
for ds in ndb.scenarios[1]["database"]:
    if ds["name"] == ("carbon dioxide, captured from atmosphere, with a solvent-based direct air capture system, 1MtCO2, with waste heat, and grid electricity",
                            "carbon dioxide, captured from atmosphere"):
            print(ds["name"], ds["location"])

In [ ]:
for ds in ndb.scenarios[1]["database"]:
    if "carbon dioxide, captured" in ds["name"]:
        print(ds["name"], ds["location"])

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU", year=2050)

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets.sel(region="WEU")

In [ ]:
ndb.update_electricity()

In [ ]:
ndb.scenarios[0]["iam data"].fuel_markets

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.update_all()

In [ ]:
ndb.generate_scenario_report()

In [ ]:
ndb.generate_change_report()

In [ ]:
ndb.scenarios[0]["iam data"].gains_data_IAM

In [ ]:
ndb.update_all()

In [ ]:
ndb.write_db_to_brightway(["test_dac", "test_dac2"])

In [ ]:
from premise.geomap import Geomap

In [ ]:
Geomap()

In [ ]:
Geomap(model="remind").geo.keys()

In [ ]:
ndb.scenarios[0]["iam data"].carbon_capture_rate.sel(variables="steel")

In [ ]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["name"].startswith("CO2 capture, at cement production plant"):
            print(e["name"], e.get("location"), e["amount"])
            print(ds["name"])
            print("--------------------")